<a href="https://colab.research.google.com/github/JacopoKahl/gan_mnist/blob/main/GAN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Generative Adversial Network

Fully-connected networks (generator and discriminator) as thew original implementation (2014)

Example on the MNIST dataset

##Imports

In [1]:
#Keras imports
from keras.models import Sequential #Sequential model 
from keras.layers import Dense, Dropout

from keras.layers.advanced_activations import LeakyReLU #advanced LeakyRelu activation layer

#Dataset
from keras.datasets import mnist

import numpy as np #import numpy library

#Visualization 
import matplotlib.pyplot as plt
from keras.utils.vis_utils import plot_model

##Load data

In [2]:
(X_train, Y_train), (X_test, Y_test) = mnist.load_data() #load dataset

#Normalization of the dataset
X_train = (X_train.astype(np.float32) - 127.5) /127.5 #normaliza pixel values in the [-1,1] interval
X_test = (X_test.astype(np.float32) - 127.5) /127.5 #normaliza pixel values in the [-1,1] interval

11493376/11490434 [==============================] - 0s 0us/step
